In [1]:
import warnings 
warnings.simplefilter('ignore')

import os
import gc
import csv
import json
import copy
import random

import numpy as np
import pandas as pd
from tqdm import tqdm
import torch.nn.functional as F

%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

from transformers import T5ForConditionalGeneration 

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup, AutoConfig
import transformers

from evaluate import CiderD

In [2]:
def split_data(data_path='./data/train.csv'):
    if not os.path.exists('./data/test.csv'):
        df = pd.read_csv(data_path)
        train_df = df[:18000].reset_index(drop=True)
        test_df = df[18001:].reset_index(drop=True)
        
        train_df.to_csv('./data/train_df.csv', header=None)
        test_df.to_csv('./data/test_df.csv', header=None)
split_data()

In [3]:
class Config:
    def __init__(self):
        super(Config, self).__init__()
        
        self.SEED = 42
        self.tokenizer_path = 'bert_model/nezha-cn-base'
        self.MODEL_PATH = './pretrain/pretrained_0.15_dual/checkpoint-20000'
        
        # data
        self.max_length = 256 #句子最大长度
        self.batch_size = 32
        
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.full_finetuning = True
        self.lr = 3e-5
        self.optimizer = 'AdamW'
        self.n_warmup = 0
        self.save_best_only = True
        
        self.multi_gpu = False
        self.attack = 'pgd'

        #self.ema = True
        self.flooding = False
        self.loss_func = 'ce' # ce dice fl
        self.epochs = 20
config = Config()

In [4]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


np.random.seed(config.SEED)
seed_everything(seed=config.SEED)

In [5]:
class GAIIC_Dataset(Dataset):
    def __init__(self, data_file, input_len, output_len, sos_id=1, eos_id=2, pad_id=0):
        super(GAIIC_Dataset, self).__init__()
        
        with open(data_file, 'r') as fp:
            reader = csv.reader(fp)
            self.samples = [row for row in reader]
            
        self.input_len   = input_len
        self.output_len  = output_len
        self.sos_id      = sos_id
        self.eos_id      = eos_id
        self.pad_id      = pad_id
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        source = [int(x) for x in self.samples[index][1].split()]
        
        if len(source) < self.input_len:
            source.extend([self.pad_id] * (self.input_len - len(source)))
        
        # for test
        if len(self.samples[index]) < 3:
            return np.array(source)[:self.input_len]
        
        target = [self.sos_id] + [int(x) for x in self.samples[index][2].split()] + [self.eos_id]
        if len(target) < self.output_len:
            target.extend([self.pad_id] * (self.output_len - len(target)))
            
        return  np.array(source)[:self.input_len], np.array(target)[:self.output_len]

In [6]:
class T5_Model(nn.Module):
    def __init__(self):
        super(T5_Model, self).__init__()

        self.model = T5ForConditionalGeneration.from_pretrained('t5-small')
        
        self.PAD_ID = 0
        self.SOS_ID = 1
        self.EOS_ID = 2
        
        self.register_buffer('true_rep', torch.tensor(1.0))
        self.register_buffer('false_rep', torch.tensor(0.0))

    def forward(self, source_tokens, target_tokens=None, max_length=150, generate=False):
        
        if target_tokens is not None:
            feed_tensor = target_tokens[:, :-1]
            target_tensor = target_tokens.detach().clone()[:, 1:]
            input_pad_mask = torch.where(source_tokens!=self.PAD_ID, self.true_rep, self.false_rep)
            output_pad_mask = torch.where(target_tensor!=self.PAD_ID, self.true_rep, self.false_rep)
            target_tensor[target_tensor==self.PAD_ID]=-100
        
        if not generate:
            outputs = self.model(
                            input_ids = source_tokens,
                            attention_mask=input_pad_mask,
                            decoder_input_ids=feed_tensor,
                            use_cache=None,
                            return_dict=False
                )
            return outputs[0], target_tensor
        else:
            outputs = self.model.generate(
                            input_ids = source_tokens,
                            max_length=max_length,
                )
            
            return outputs

In [7]:
def array2str(arr):
    out = ''
    for i in range(len(arr)):
        if arr[i]==0 or arr[i]==2:
            break
        if arr[i]==1:
            continue
        out += str(int(arr[i])) + ' '
    if len(out.strip())==0:
        out = '0'
        
    return out.strip()

In [8]:
def val_fn(model, valid_dataloader):
    val_loss = 0

    model.eval()
    res, gts = [], {}
    tot = 0
    device = 'cuda'
    for batch_idx, (input_ids, target_ids) in enumerate(tqdm(valid_dataloader)):
        
        input_ids  = input_ids.to(device)
        target_ids = target_ids.cpu().numpy()
        
        pred = model(source_tokens=input_ids, 
                     max_length=150,
                     generate=True).cpu().numpy()
        
        for i in range(pred.shape[0]):
            res.append({'image_id':tot, 'caption': [array2str(pred[i])]})
            gts[tot] = [array2str(target_ids[i][1:])]
            tot += 1
            
    CiderD_scorer = CiderD(df='corpus', sigma=15)
    cider_score,  = CiderD_scorer.compute_score(gts, res)
    print("cider_score:", cider_score, cider_scores)
    
    model.train()

    return cider_score, cider_scores

In [9]:
def test(model, test_dataloader):
    val_loss = 0

    model.eval()
    res = []
    tot = 0
    device = 'cuda'
    for batch_idx, (input_ids, target_ids) in enumerate(tqdm(test_dataloader)):
        
        input_ids  = input_ids.to(device)
        target_ids = target_ids.cpu().numpy()
        
        pred = model(source_tokens=input_ids, 
                     max_length=150,
                     generate=True).cpu().numpy()
        
        for i in range(pred.shape[0]):
            res.append({'image_id':tot, 'caption': [array2str(pred[i])]})
            gts[tot] = [array2str(target_ids[i])]
            tot += 1

    return res

In [10]:
def train_fn(model, train_dataloader, criterion, optimizer, scheduler=None, epoch=0):
    
    train_loss = 0
    model.train()
    device = 'cuda'

    for batch_idx, (input_ids, target_ids) in enumerate(tqdm(train_dataloader, desc='Epoch ' + str(epoch))):
        
        input_ids = input_ids.to(device)
        target_ids = target_ids.to(device)
        
        optimizer.zero_grad()
        
        logits, targets = model(source_tokens=input_ids, target_tokens=target_ids)
        
        loss = criterion(logits.view(-1, logits.shape[-1]), 
                        targets.contiguous().view(-1))
        
        train_loss += loss.item()
        
        loss.backward()
        
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f"Epoch {epoch}, batch {batch_idx}, loss {loss.item()}")
        if scheduler:
            scheduler.step()
            
    avg_train_loss = train_loss / len(train_dataloader)
    
    print(f"Epoch {epoch}: average loss = {avg_train_loss}")

In [11]:
def main():
    print("Start Training")
    
    train_dataset = GAIIC_Dataset('./data/train_df.csv', 150, 150)
    val_dataset = GAIIC_Dataset('./data/test_df.csv', 150, 150)
    test_dataset = GAIIC_Dataset('./data/preliminary_a_test.csv', 150, 150)
    
    train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    val_dataloader  = DataLoader(val_dataset, batch_size=64, shuffle=False)
    test_dataloader  = DataLoader(test_dataset, batch_size=64, shuffle=False)
    
    model = T5_Model().to('cuda')
    
    optimizer = torch.optim.AdamW(model.parameters(),lr=2e-5)
    
    criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)
    
    epochs = 10
    bst_val_cider = 0
    for epoch in range(epochs):
        train_fn(model, train_dataloader, criterion, optimizer, scheduler=None, epoch=epoch)
        
        cider_score, cider_scores = val_fn(model, val_dataloader)
        
        if cider_score >= bst_val_cider:
            best_model_wts = copy.deepcopy(model.state_dict())
    
    print(f"Best val cider: {bst_val_cider}")
    model.load_state_dict(best_model_wts)

    test_outputs = test(test_dataloader, model)
    
    return test_outputs

In [12]:
test_outputs = main()

Start Training


Epoch 0:   1%|          | 1/141 [00:01<03:16,  1.40s/it]

Epoch 0, batch 0, loss 16.51783561706543


Epoch 0:  72%|███████▏  | 101/141 [01:23<00:32,  1.22it/s]

Epoch 0, batch 100, loss 9.202470779418945


...

 97%|█████████▋| 31/32 [00:37<00:01,  1.29s/it]

0
34 12 38 41 30 623 13 248 111 106 410 25 10 34 12 38 497 17 27 22 96 17 16 48 17 1204 131 17 89 65 69 70 11 58 51 36 82 17 91 30 15 13 159 54 73 23 21 20 18 10 22 12 33 91 72 78 11 33 122 61 24 10 22 12 67 33 148 171 82 11 33 119 321 170 179 11 22 12 33 119 17 143 175 105 99 26 37 72 78 11 34 12 38 41 17 210 143 234 761 179 
0
34 12 48 17 115 19 166 11 34 12 48 16 142 16 160 49 77 75 100 92 177 11 167 67 27 160 38 49 13 52 43 89 99 25 10 129 136 45 83 111 220 168 1036 11 129 264 195 213 317 352 11 136 30 357 358 99 16 335 43 99 23 21 168 18 10 129 120 239 31 29 36 314 11 346 221 51 36 45 10 22 12 291 193 192 261 246 168 1230 11 23 21 195 213 11 97 141 126 265 10 34 12 48 49 50 30 40 13 182 89 10 34 12 38 41 19 48 106 256 33 346 439 404 19 28 351 13 139 230 236 10 
0
22 12 74 71 64 27 279 288 285 56 13 83 112 52 708 44 23 21 25 10 24 42 1045 37 51 68 60 10 14 32 101 102 15 13 18 10 14 39 16 14 47 15 13 20 18 563 79 46 50 19 15 13 20 10 34 12 163 49 167 67 252 943 201 10 
0
22 12 16 85

100%|██████████| 32/32 [00:38<00:00,  1.19s/it]

0
33 19 15 13 29 140 151 27 19 166 144 145 10 181 206 77 75 100 13 31 29 20 18 23 21 25 10 
0
22 62 80 17 89 65 69 70 11 22 12 38 41 17 16 96 17 27 17 30 13 23 21 36 53 25 11 38 41 17 17 143 298 209 11 15 13 89 186 113 142 11 17 115 19 28 13 1008 29 140 151 144 145 10 22 12 33 148 27 33 91 149 171 82 10 22 12 33 119 298 209 11 33 24 122 35 152 10 
0
14 58 201 116 88 175 226 1234 12 48 13 83 52 43 44 23 21 54 509 97 147 126 10 14 32 102 15 13 31 29 20 18 11 24 42 26 704 61 48 19 986 21 139 213 73 158 44 11 159 49 86 308 10 
0
22 12 217 320 128 217 143 16 217 368 273 72 78 11 239 323 252 266 290 126 231 10 33 217 968 115 77 75 100 15 13 36 82 11 22 12 217 249 208 231 51 68 60 11 22 12 154 187 137 559 56 13 31 29 92 45 292 293 26 10 50 71 28 15 13 29 140 144 145 10 
0
22 12 14 32 94 109 28 16 12 74 71 64 56 13 66 55 44 23 21 25 11 14 32 16 39 27 14 47 29 46 69 70 11 24 42 26 37 61 24 10 265 46 62 1064 15 13 31 95 19 28 20 18 10 80 33 17 15 20 18 10 
0
22 12 129 160 27 552 136 72 78 11 45 

ValueError: too many values to unpack (expected 1)